# Using IHttpClientFactory and Duende AccessToken Management

This code sample demonstrates how to configure an ASP.NET Core host to use Duende's client credentials token management. It shows how to register a client with the required token endpoint, client ID, and secret, and how to set up an HttpClient that automatically manages access tokens.

For a detailed explanation of the code flow, see the [Web server host tutorial](../../../tutorials/server-to-server/web-server-host).

## Steps

1. Start the API using the [WebAPI sample](https://github.com/FHIDev/Fhi.AuthExtensions/tree/main/samples/Fhi.Samples.WebApi).
2. Run the code below with .NET Interactive in VS Code or Jupyter Lab.

In [ ]:
#!csharp
#r "nuget: Duende.AccessTokenManagement, 3.2.0"
#r "nuget: Microsoft.Extensions.Hosting, 9.0.5"
#r "nuget: Microsoft.Extensions.Caching.Memory, 9.0.5"

using Microsoft.Extensions.Hosting;
using Duende.AccessTokenManagement;
using Microsoft.Extensions.DependencyInjection;
using System.Net.Http;

async Task RunAsync()
{
    var builder = Host.CreateDefaultBuilder();

    builder.ConfigureServices(services =>
    {
        services
            .AddClientCredentialsTokenManagement()
            .AddClient("m2m", options =>
            {
                options.TokenEndpoint = "https://demo.duendesoftware.com/connect/token";
                options.ClientId = "m2m";
                options.ClientSecret = "secret";
                options.Scope = "api";
            });

        services.AddClientCredentialsHttpClient("m2m", "m2m", client =>
        {
            client.BaseAddress = new Uri("https://localhost:7150");
        });

        services.AddDistributedMemoryCache();
        services.AddTransient<ITestService, TestService>();
    });

    var host = builder.Build();
    
    using var scope = host.Services.CreateScope();
    var testService = scope.ServiceProvider.GetRequiredService<ITestService>();
    await testService.Get();
}

interface ITestService
{
    public Task Get();
}

// <summary>
// A service that uses the registerd HttpClient with HttpClientFactory to make requests.
class TestService : ITestService
{
    private readonly IHttpClientFactory _factory;
    public TestService(IHttpClientFactory factory)
    {
        _factory = factory;
    }
    
    public async Task Get()
    {
        var client = _factory.CreateClient("m2m");
        Console.WriteLine("Base address: " + client.BaseAddress);
         var response = await client.GetAsync("api/v1/integration/health-records");
        Console.WriteLine(response);
    }
}

await RunAsync();